In [4]:
import pandas as pd
import numpy as np
import os
import gc
import copy
import json
import sys
from pathlib import Path
from datetime import datetime, timedelta, date
import time
from dateutil.relativedelta import relativedelta 

import pyarrow.parquet as pq
import pyarrow as pa

from tqdm import tqdm

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

pd.options.display.float_format = '{:,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
random_seed=1
pl.seed_everything(random_seed)

Global seed set to 1


1

In [7]:
from datasets import load_dataset

In [9]:
train_file = "../MIRI/train.jsonl"

In [10]:
%%time
dataset = load_dataset("json", data_files= str(train_file))

Using custom data configuration default-16a36c29b56829d4


Computing checksums of downloaded files. They can be used for integrity verification. You can disable this by passing ignore_verifications=True to load_dataset


Computing checksums: 100%|##########| 1/1 [00:09<00:00,  9.21s/it]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/tiami/.cache/huggingface/datasets/json/default-16a36c29b56829d4/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: total: 4min 50s
Wall time: 43.1 s


In [11]:
dataset.shape

{'train': (12899779, 2)}

In [12]:
train_size = dataset.shape['train'][0]
print(f'Num. of samples in train dataset: {train_size:,}')

Num. of samples in train dataset: 12,899,779


In [13]:
%%time
session_stats = []
for i in range(100000):
    cnt = len(dataset['train'][i]['events'])
    session_stats.append([dataset['train'][i]['session'], cnt])

CPU times: total: 1min 45s
Wall time: 1min 45s


In [15]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['session', 'events'],
        num_rows: 12899779
    })
})

In [17]:
df = pd.DataFrame(list(dataset.items()), columns=['session', 'events'])

In [18]:
df.head()

,session,events
0,train,"({'session': 0, 'events': [{'aid': 1517085, 't..."
